### Conformal Prediction desde ZERO
Ejemplo entrenando modelo para forecasting probabilistico utilizando conformal prediction DESDE CERO sin utilizar libreria de mlforecast

Adicionalmente comparar los resulados de conformal prediction propio con el que retorna la librería (adaptando el ejemplo https://nixtlaverse.nixtla.io/mlforecast/docs/how-to-guides/prediction_intervals.html)

### Cargar data m4 - 6 datasets

In [52]:
import pandas as pd
from utilsforecast.plotting import plot_series
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from mlforecast import MLForecast
from mlforecast.utils import PredictionIntervals
import numpy as np

In [13]:
train = pd.read_csv('https://auto-arima-results.s3.amazonaws.com/M4-Hourly.csv')
test = pd.read_csv('https://auto-arima-results.s3.amazonaws.com/M4-Hourly-test.csv')

In [14]:
n_series = 1 # samplear solo una serie
uids = train['unique_id'].unique()[:n_series] # select first n_series of the dataset
train = train.query('unique_id in @uids')
test = test.query('unique_id in @uids')

In [15]:
train.head()

,unique_id,ds,y
0,H1,1,605.0
1,H1,2,586.0
2,H1,3,586.0
3,H1,4,559.0
4,H1,5,511.0


In [16]:
test.head()

,unique_id,ds,y
0,H1,701,619.0
1,H1,702,565.0
2,H1,703,532.0
3,H1,704,495.0
4,H1,705,481.0


### 2. Entender probabilistic forecasting usando nixtla

In [39]:
# crear objeto de mlforecast para hacer forecast
mlf = MLForecast(
    models = LinearRegression(),
    freq = 1,
    lags = [1, 2, 3]
)
mlf

MLForecast(models=[LinearRegression], freq=1, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [40]:
# entrenar modelo
mlf.fit(
    train,
    prediction_intervals=PredictionIntervals(n_windows=5, h=1),
)

MLForecast(models=[LinearRegression], freq=1, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [76]:
# hacer forecast
levels = [5, 50, 80, 95]
forecasts = mlf.predict(1, level=levels)

In [77]:
forecasts

,unique_id,ds,LinearRegression,LinearRegression-lo-95,LinearRegression-lo-80,LinearRegression-lo-50,LinearRegression-lo-5,LinearRegression-hi-5,LinearRegression-hi-50,LinearRegression-hi-80,LinearRegression-hi-95
0,H1,701,636.090121,602.88092,604.536773,617.279227,631.330918,640.849323,654.901014,667.643468,669.299321


In [115]:
#### REVISAR DIFERENCIA DE INTERVALOS CON RESPECTO A PREDICCIÓN PUNTUAL
print('low_95-puntual', forecasts['LinearRegression-lo-95'] - forecasts['LinearRegression'])
print('high_95-puntual', forecasts['LinearRegression-hi-95'] - forecasts['LinearRegression'])
print('low_50-puntual', forecasts['LinearRegression-lo-50'] - forecasts['LinearRegression'])
print('high_50-puntual', forecasts['LinearRegression-hi-50'] - forecasts['LinearRegression'])

low_95-puntual 0   -33.209201
dtype: float64
high_95-puntual 0    33.209201
dtype: float64
low_50-puntual 0   -18.810894
dtype: float64
high_50-puntual 0    18.810894
dtype: float64


### 3. Entrenar forecast probabilistico - simple - own

#### Forecast Puntual

In [45]:
# obtener dataframe X e y para entrenar
X_train = mlf.preprocess(train)[['lag1', 'lag2', 'lag3']]
y_train = mlf.preprocess(train)[['y']]

In [46]:
# entrenar modelo
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [57]:
# generar instancia para predecir
instance = pd.DataFrame(np.expand_dims(np.flip(train.tail(3)['y'].values), axis = 0),
                        columns = ['lag1', 'lag2', 'lag3']
                       )
instance

,lag1,lag2,lag3
0,684.0,739.0,752.0


In [80]:
# precedir forecast puntual
y_fcst = model.predict(instance)
y_fcst

array([[636.09012055]])

#### Tranformar a forecast probabilistico - forma simple conformal predictions
Se obtienen los errores y se calcula la distribución de los errores.

Luego, agregarlos como intervalos

Solo usando numpy y pandas

In [63]:
# obtener dataset de calibración (para este ejemplo se usan los mismos datos de entrenamiento)
y_calib_pred = model.predict(X_train)
y_calib = y_train

In [64]:
# calcular errores de conformidad
errors = np.abs(y_calib_pred - y_calib)

In [67]:
# Obtener el percentil deseado del error (e.g., 95% de confianza)
alpha = 0.05
q = np.quantile(errors, 1 - alpha)
q

31.748113803820914

In [79]:
# generar intervalos donde dentro esté el 50% de los datos, 80% y 95% de los datos
errors_p50 = errors.quantile(0.05)
errors_p80 = errors.quantile(0.80)
errors_p95 = errors.quantile(0.95)

In [95]:
# agregar intervalos
y_fcst_lo_95 = (y_fcst[0] - errors_p95).values[0]
y_fcst_lo_80 = (y_fcst[0] - errors_p80).values[0]
y_fcst_lo_50 = (y_fcst[0] - errors_p50).values[0]
y_fcst_hi_50 = (y_fcst[0] + errors_p50).values[0]
y_fcst_hi_80 = (y_fcst[0] + errors_p80).values[0]
y_fcst_hi_95 = (y_fcst[0] + errors_p95).values[0]

In [100]:
# guardar prediccion en fcst
pd.DataFrame([[
    y_fcst[0][0],
    y_fcst_lo_95,
    y_fcst_lo_80,
    y_fcst_lo_50,
    y_fcst_hi_50,
    y_fcst_hi_80,
    y_fcst_hi_95
]],
    columns = ['LinearRegression', 'LinearRegression-lo-95','LinearRegression-lo-80', 'LinearRegression-lo-50',
            'LinearRegression-hi-50', 'LinearRegression-hi-50', 'LinearRegression-hi-95'
              ]
)

,LinearRegression,LinearRegression-lo-95,LinearRegression-lo-80,LinearRegression-lo-50,LinearRegression-hi-50,LinearRegression-hi-50,LinearRegression-hi-95
0,636.090121,604.342007,617.088964,635.438148,636.742093,655.091277,667.838234


In [117]:
# contrastar con fcst de nixtla
forecasts

,unique_id,ds,LinearRegression,LinearRegression-lo-95,LinearRegression-lo-80,LinearRegression-lo-50,LinearRegression-lo-5,LinearRegression-hi-5,LinearRegression-hi-50,LinearRegression-hi-80,LinearRegression-hi-95
0,H1,701,636.090121,602.88092,604.536773,617.279227,631.330918,640.849323,654.901014,667.643468,669.299321


### OBSERVACIÓN:
Se puede ver que son valores distintos, ahora la pregunta es, cuál es la metodología que utiliza nixtla para generar el intervalo

In [68]:
alpha = 0.95
q = np.quantile(errors, 1 - alpha)
q

0.6519726300529788

In [73]:
errors.sort_values(by = 'y')

,y
492,0.004334
667,0.033915
570,0.070720
296,0.082677
224,0.084231
...,...
42,47.454354
290,48.126396
613,48.717238
446,50.845736


y    31.748114
Name: 0.95, dtype: float64